<a href="https://colab.research.google.com/github/yamadashamoji/00_tools/blob/main/pdf_to_markdown_extended.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title PDFをMarkdownに変換する

# --- セル 1: ライブラリのインストール ---
!pip install -q pymupdf

# --- セル 2: 必要なライブラリのインポート ---
import fitz  # PyMuPDF
import os
import re
from google.colab import files
from pathlib import Path
from collections import defaultdict

# --- セル 3: PDFファイルをアップロード ---
print("PDFファイルをアップロードしてください。")
uploaded = files.upload()

pdf_path = list(uploaded.keys())[0]
print(f"アップロード完了: {pdf_path}")

# --- セル 4: PDFの解析と抽出 ---
doc = fitz.open(pdf_path)

text_blocks = []
images = []
tables = []
footnotes = []
links = []

output_img_dir = Path("pdf_images")
output_img_dir.mkdir(exist_ok=True)

for page_num, page in enumerate(doc):
    blocks = page.get_text("dict")["blocks"]
    links.extend(page.get_links())

    for b in blocks:
        if b["type"] == 0:  # テキスト
            text_blocks.append((page_num, b))
        elif b["type"] == 1:  # 画像
            pix = page.get_pixmap(matrix=fitz.Matrix(2, 2))
            image_path = output_img_dir / f"image_{page_num+1}_{len(images)+1}.png"
            pix.save(str(image_path))
            images.append(image_path)

        # 擬似的な「表」認識（横方向に均等なセル配置と仮定）
        if b.get("lines") and len(b["lines"]) > 1:
            row_texts = []
            for line in b["lines"]:
                spans = line["spans"]
                texts = [span["text"] for span in spans]
                if len(texts) > 1:
                    row_texts.append(texts)
            if len(row_texts) > 1:
                tables.append(row_texts)

print(f"抽出結果: テキスト {len(text_blocks)}件, 画像 {len(images)}枚, 表 {len(tables)}件, リンク {len(links)}件")

# --- セル 5: Markdown変換処理（見出し判定/リンク/脚注） ---
markdown_lines = []
footnote_counter = 1
footnote_dict = {}

for page_num, block in text_blocks:
    for line in block["lines"]:
        if not line["spans"]:
            continue
        span = line["spans"][0]
        text = " ".join(s["text"] for s in line["spans"]).strip()
        if not text:
            continue

        # 見出し判定（フォントサイズ＋ボールド）
        font_size = span["size"]
        font_flags = span["flags"]  # 2: bold, 4: italic
        is_bold = font_flags & 2 != 0

        if font_size > 18 or (font_size > 14 and is_bold):
            markdown_lines.append(f"# {text}")
        elif font_size > 14:
            markdown_lines.append(f"## {text}")
        elif is_bold:
            markdown_lines.append(f"**{text}**")
        else:
            # 脚注形式の検出（例："[1]"や"※"など）
            if re.match(r"^\[\d+\]|\※", text):
                label = f"[^note{footnote_counter}]"
                footnote_dict[label] = text
                markdown_lines.append(label)
                footnote_counter += 1
            else:
                markdown_lines.append(text)
    markdown_lines.append("")

# 画像挿入
for image_path in images:
    markdown_lines.append(f"![image]({image_path})")
    markdown_lines.append("")

# 表のMarkdown形式への変換
for table in tables:
    if not table or not table[0]:
        continue
    header = table[0]
    separator = ['---'] * len(header)
    markdown_lines.append("| " + " | ".join(header) + " |")
    markdown_lines.append("| " + " | ".join(separator) + " |")
    for row in table[1:]:
        markdown_lines.append("| " + " | ".join(row) + " |")
    markdown_lines.append("")

# リンクのMarkdown形式への変換
if links:
    markdown_lines.append("## 参考リンク")
    for l in links:
        uri = l.get("uri")
        if uri:
            markdown_lines.append(f"- [{uri}]({uri})")

# 脚注の追加
if footnote_dict:
    markdown_lines.append("## 脚注")
    for label, content in footnote_dict.items():
        markdown_lines.append(f"{label}: {content}")

# --- セル 6: Markdownファイルとして保存＆ダウンロード ---
md_filename = Path(pdf_path).stem + "_converted.md"
with open(md_filename, "w", encoding="utf-8") as f:
    f.write("\n".join(markdown_lines))

print(f"Markdownファイルを生成しました: {md_filename}")
files.download(md_filename)

# 画像もダウンロード
for img in images:
    files.download(str(img))

# --- セル 7: 完了メッセージ ---
print("✅ 変換完了！Markdownファイルと画像がダウンロード可能です。")



---

### 説明（Markdown形式）

---

## 📘 このノートブックについて

このGoogle Colabノートブックは、**PDFファイルをMarkdown形式へ変換**するツールです。以下の機能を備えています：

* ✅ **テキスト抽出と構造化（見出し・太字・本文）**
* 🖼 **画像の抽出とMarkdown埋め込み**
* 📊 **表検出とMarkdownテーブル変換**
* 🔗 **PDF内のリンク抽出**
* 📝 **脚注処理（番号付き脚注対応）**

---

## 👤 利用者タイプ別ガイド

### ▶️ 変換だけしたい人向け

1. このノートブックをColabで開いてください。
2. PDFファイルをアップロードする指示が出たらファイルを選択してください。
3. 自動で変換され、Markdownファイルと画像がダウンロード可能になります。

**特別な知識は不要です！**

---

### 🛠 改修・再利用したい人向け

このノートブックは以下のようにモジュール構造的に整理されています：

| セル  | 内容                        |
| --- | ------------------------- |
| セル1 | PyMuPDFのインストール            |
| セル2 | 必要なライブラリのインポート            |
| セル3 | ファイルアップロード処理              |
| セル4 | PDF内容の抽出処理（テキスト・画像・表・リンク） |
| セル5 | Markdown形式への変換ロジック        |
| セル6 | Markdownファイルと画像の保存・ダウンロード |
| セル7 | 完了メッセージ表示                 |

**改修ポイント例：**

* 表抽出アルゴリズムの精度向上（現在は行数と列数で判定）
* フォントの種類（name）も見てスタイルを補強
* H1〜H3の変換ルール変更
* 画像の保存パスやファイル形式の調整

---